In [257]:
import iparser
import importlib
import pickle
import spacy
import tqdm
import utils 
from spacy.lang.en import English
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

In [286]:
importlib.reload(utils)
pdict = utils.patterns_dict

## Топ профессий:

 

* Java разработчик/Software engineer
* Front-end разработчик/Web разработчик
* Продуктовый дизайнер
* Системный аналитик
* Архитектор систем
* Agile coach (под вопросом?)
* Data Scientist
* Data engineer
* Бизнес-аналитик
* Финансовый аналитик
* Кредитный аналитик
* Юрист (корпоративный, судебный)
* Маркетолог
 

 

In [2]:
with open("london.pkl", "rb") as f:
    vacancy_dict = pickle.load(f)

In [72]:
# list(vacancy_dict.keys())

In [4]:
for k,v in vacancy_dict.items():
    print(f"{k}:{len(v)}")

Java Developer :716
Software engineer:866
Web Developer:738
Front-end Developer:547
Product Designer:689
System analyst:775
System architect:673
Agile coach:227
Data Scientist:385
Data engineer:787
Business-analyst:353
Financial analyst:387
Credit analyst:166
Corporate lawyer:298
Judicial Lawyer:14
Marketing Manager:670
Marketing Specialist:700


In [5]:
nlp = English()

In [218]:
s = vacancy_dict['Data Scientist'][1]
doc = nlp(s)
doc

Data Scientist- Permanent - London
£50-70k (dependant on experience)
A highly successful and scaling start-up is experiencing a rapid period of growth due to an influx of projects from clients. Therefore, they require a number of Data Scientists, ranging from Mid-Senior.
Day to day:
Developing algorithms to reduce travel delays and improving 3rd party network capacity
Performing exploratory data analysis of train movement and signalling system data across a key UK transport network. Designing experiments and interpret the results to draw detailed and actionable conclusions
Working closely with 3rd party operational performance teams to develop algorithms for extracting valuable insights from operational data
Working with our software engineers and data scientists on the team to rapidly build, test and deploy robust algorithms and data analytic solutions.
Required Skills:
At least 3+ years R&D experience in industry or academia. Holding at least an BSc, and preferably a MSc/ PhD in a sc

In [249]:
reqs = []
for s in tqdm.notebook.tqdm(vacancy_dict['Data Scientist']):
    doc = nlp(s)
    res = requirements_extractor(doc)
    if res:
        if len(res) > 4:
#             print("_"*10)
#             print(res)
#             print(len(res))
            reqs.append(res)

In [251]:
len(reqs)

73

In [194]:
reqs[5]

requirements of the role and your motivation behind applying.


In [166]:
requirements_extractor(doc)

In [254]:
s = vacancy_dict['Data Scientist'][30]
doc = nlp(s)
# doc

In [ ]:
matcher = Matcher(nlp.vocab)

pattern = [
    {'LEMMA':'requirement'},
    {'LEMMA':'skills'},
    {'LEMMA':'experience'},
]

In [261]:
for el in pdict.values():
    matcher.add('REQ_PATTERN', None, el)

In [150]:
# doc[502:]

In [116]:
c = 0
for el in doc[502:]:
    if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
        print(doc[el.i+1], el.i+1)
        c = el.i + 1
        break


Accountabilities 600


In [124]:
doc[matches[0][1]:c]

Requirements:
A Finance/Economics/Business/Mathematics graduate
A demonstrated interest and knowledge in the financial markets
A proven record of outstanding achievement in academic and extracurricular activities
An independent thinker and decision maker who is able to contribute effectively to the team's success
Superior interpersonal and analytical skills
Ability to meet the challenges presented by a stressful, constantly changing work environment.
Ideally available to start asap – individual will be expected to go to Toronto for training 13th August for 2 wks.






In [288]:
matcher = Matcher(nlp.vocab)
for el in pdict.values():
    matcher.add('REQ_PATTERN', None, el)

In [308]:
def requirements_extractor(doc, matcher=matcher):
    endpos = None
    matches = matcher(doc)
        
    if len(matches) > 0 :
        startpos = matches[0][1]
        for el in doc[startpos:]:
            if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
#                 print(doc[el.i+1], el.i+1)
                endpos = el.i + 1
                break
    if endpos:
        return doc[startpos:endpos]
    else:
        return None
    

In [309]:
reqs = []
for s in tqdm.notebook.tqdm(vacancy_dict['Data Scientist']):
    doc = nlp(s)
    res = requirements_extractor(doc)
    if res:
        if len(res) > 4:
#             print("_"*10)
#             print(res)
#             print(len(res))
            reqs.append(res)
print(f"Удалось извлечь потенциальных скилов {len(reqs)} шт.")


Удалось извлечь потенциальных скилов 101 шт.


In [305]:
reqs[29]

Requirements:
Must Have:


In [148]:
importlib.reload(iparser)

parsed_dict = {"Agile coach": iparser.IndeedParser(keywords="Credit analyst", region="London")}
for prof_parser in parsed_dict.values():
    prof_parser.start()
for prof_parser in parsed_dict.values():
    prof_parser.join()

vacs_dict = {}
for k, v in parsed_dict.items():
    vacs_dict[k] = v.vacancy_texts

https://www.indeed.co.uk/jobs?q=Credit+analyst&l=London


In [161]:
bv = vacs_dict['Agile coach'][2]

In [162]:
docbv = nlp(bv)

In [163]:
docbv

Global Markets (Credit Products), High Yield Sector Strategist, VP, London, #152732
United Kingdom-London-London | Full-time (FT) | Global Markets | Job ID 152732
Credit Suisse is a leading global wealth manager with strong investment banking capabilities. Headquartered in Zurich, Switzerland, we have a global reach with operations in about 50 countries and employ more than 45,000 people from over 150 different nations. Embodying entrepreneurial spirit, Credit Suisse delivers holistic financial solutions to our clients, including innovative products and specially tailored advice. Striving for quality and excellence in our work, we recognize and reward extraordinary performance among our employees, provide wide-ranging training and development opportunities, and benefit from a diverse range of perspectives to create value for our clients, shareholders and communities. We are Credit Suisse.
We Offer

The Credit Suisse leverage finance trading desk is looking for a desk analyst at the VP/

In [157]:
for token in docbv:
    print(token.text)

Our
client
is
an
opportunistic
debt
and
equity
investor
.
Operating
across
the
capital
structure
,
they
are
seeking
candidates
with
experience
across
leveraged
finance
style
transactions
both
from
a
research
and
execution
background
.
On
offer
will
be
the
opportunity
to
work
in
a
busy
,
highly
deal
driven
environment
,
across
various
sectors
,
with
an
entrepreneurial
business
team
that
adapts
its
model
to
suit
changing
conditions
constantly
.
The
role
will
encompass
,
debt
finance
,
direct
lending
,
and
equity
investments
,
and
perhaps
secondary
loan
investing
.
You
will
be
intellectually
curious
,
dynamic
and
have
a
natural
ability
for
numbers
.
Ideally
you
will
have
also
had
deep
experience
managing
stakeholders
and
working
within
deal
teams
on
highly
complex
leveraged
transactions
.
For
more
information
about
the
firm
,
culture
and
opportunity
please
reach
out
with
an
updated
copy
of
CV
to
Eversearch.(All
interviews
are
being
conducted
remotely
and
our
client
has
confirmed
progressi